In [1]:
# ✅ Instalação (executar uma vez)
%pip install -U langchain langchain-community langchain-ollama langchain-chroma pymupdf ccxt tqdm


Note: you may need to restart the kernel to use updated packages.


In [27]:
# ✅ Carregamento do PDF
import pathlib
from langchain.document_loaders import PyMuPDFLoader

pdf_path = pathlib.Path(r"C:\Users\paulo\Downloads\Tradedocuments.pdf")
loader = PyMuPDFLoader(str(pdf_path))
documents = loader.load()
print(f"📄 Total de páginas: {len(documents)}")

📄 Total de páginas: 2334


In [29]:
# ✅ Separação em chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter


splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = splitter.split_documents(documents)
print(f"📚 Total de chunks: {len(docs)}")

📚 Total de chunks: 5033


In [30]:
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = OllamaEmbeddings(model="llama3")
db = Chroma.from_documents(docs, embedding=embeddings, persist_directory="chroma_db")
db.persist()
print("✅ Embeddings salvos")

✅ Embeddings salvos


In [31]:
# ✅ Função para consultar a LLM com RAG
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama

def consultar_modelo(pergunta: str):
    retriever = db.as_retriever()
    chain = RetrievalQA.from_chain_type(
        llm=ChatOllama(model="llama3"),
        retriever=retriever
    )
    resposta = chain.run(pergunta)
    return resposta

In [32]:
# ✅ Buscar candles da Binance
import ccxt

def buscar_candles_binance(symbol="BTC/USDT", timeframe='5m', limit=50):
    binance = ccxt.binance()
    ohlcv = binance.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    return ohlcv

In [33]:
# ✅ Converter candles em texto técnico
def gerar_texto_candle(ohlcv):
    texto = ""
    for ts, o, h, l, c, v in ohlcv[-10:]:
        texto += f"Abertura: {o:.2f}, Fechamento: {c:.2f}, Máxima: {h:.2f}, Mínima: {l:.2f}\n"
    return texto

In [34]:
!ollama pull llama3:8b

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest 
pulling 6a0746a1ec1a: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 4.7 GB                         
pulling 4fa551d4f938: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  12 KB                         
pulling 8ab4849b038c: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  254 B                         
pulling 577073ffcc6c: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  110 B                         
pulling 3f8eb4da87fa: 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [11]:
!pip install python-binance

  Using cached dateparser-1.2.1-py3-none-any.whl.metadata (29 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzlocal-5.3.1-py3-none-any.whl.metadata (7.6 kB)
Using cached dateparser-1.2.1-py3-none-any.whl (295 kB)
   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ---------------------------------------- 1.8/1.8 MB 10.9 MB/s eta 0:00:00
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzlocal-5.3.1-py3-none-any.whl (18 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2024.1
    Uninstalling pytz-2024.1:
      Successfully uninstalled pytz-2024.1


In [ ]:
from langchain_ollama import ChatOllama
from binance.client import Client
import pandas as pd
import datetime

# 🔧 Configurar conexão com Binance
API_KEY = "sua_api"
API_SECRET = "sua_secret"
client = Client(API_KEY, API_SECRET)

# 🔧 Função para buscar candles
def buscar_candles(symbol="BTCUSDT", interval="5m", limit=10):
    klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)
    df = pd.DataFrame(klines, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
    ])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms')
    return df[["timestamp", "open", "high", "low", "close"]]

# ✅ Buscar dados reais dos três tempos gráficos
df_diario = buscar_candles(interval="1d", limit=1)
df_h1     = buscar_candles(interval="1h", limit=1)
df_m5     = buscar_candles(interval="5m", limit=5)

# ✅ Simular análise técnica básica
def analisar(df, ciclo_nome):
    open_ = float(df.iloc[-1]["open"])
    close = float(df.iloc[-1]["close"])
    high  = float(df.iloc[-1]["high"])
    low   = float(df.iloc[-1]["low"])
    direcao = "Alta" if close > open_ else "Baixa"
    suporte = round(low, 2)
    resistencia = round(high, 2)
    return f"Ciclo: Canal Estreito de {direcao}\nSuporte: {suporte}\nResistência: {resistencia}"

# ✅ Gerar info com base nos dados atuais
info = f"""
--- Gráfico Diário ---
{analisar(df_diario, 'Diário')}

--- Gráfico H1 ---
{analisar(df_h1, 'H1')}

--- Gráfico M5 ---
{analisar(df_m5, 'M5')}
Número de correções: 3
Topo/Fundo Duplo: Não
Falha de Rompimento: Não
Última vela é Marubozu: Não
"""

# ✅ Gerar candles recentes formatados
candles = "\n".join([
    f"Abertura: {row.open}, Fechamento: {row.close}, Máxima: {row.high}, Mínima: {row.low}"
    for _, row in df_m5.iterrows()
])

# ✅ Criar o prompt
prompt = f"""
Sou um trader utilizando price action. Todas as decisões são feitas no gráfico M5.

{info}

Candles recentes:
{candles}

Com base nisso, responda em português e de forma organizada:
1. Devo comprar, vender ou aguardar?
2. Qual o melhor horário para entrar?
3. Onde devo posicionar o stop loss e o alvo da operação?
"""

# ✅ Consultar modelo Ollama
modelo = ChatOllama(model="llama3")
resposta = modelo.invoke(prompt)

# ✅ Exibir a resposta limpa
print("📈 Resposta da LLM:\n")
print(resposta.content)

In [35]:
!pip install MetaTrader5 pandas

In [52]:
import MetaTrader5 as mt5
import pandas as pd
import datetime
from langchain_ollama import ChatOllama

# ✅ Inicializar conexão com MetaTrader 5
if not mt5.initialize():
    raise RuntimeError("❌ Falha ao conectar ao MetaTrader 5")

# ✅ Função para buscar candles no MT5
def buscar_candles_mt5(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_M5, n=200):
    hoje = datetime.datetime.now()
    rates = mt5.copy_rates_from(symbol, timeframe, hoje, n)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.rename(columns={'time': 'timestamp', 'open': 'open', 'high': 'high', 'low': 'low', 'close': 'close'}, inplace=True)
    return df[['timestamp', 'open', 'high', 'low', 'close']]

# ✅ Buscar dados dos três tempos gráficos
df_diario = buscar_candles_mt5(timeframe=mt5.TIMEFRAME_D1, n=1)
df_h1     = buscar_candles_mt5(timeframe=mt5.TIMEFRAME_H1, n=1)
df_m5     = buscar_candles_mt5(timeframe=mt5.TIMEFRAME_M5, n=5)

# ✅ Simulação de análise técnica simples
def analisar(df, ciclo_nome):
    open_ = float(df.iloc[-1]["open"])
    close = float(df.iloc[-1]["close"])
    high  = float(df.iloc[-1]["high"])
    low   = float(df.iloc[-1]["low"])
    direcao = "Alta" if close > open_ else "Baixa"
    suporte = round(low, 2)
    resistencia = round(high, 2)
    return f"Ciclo: Canal Estreito de {direcao}\nSuporte: {suporte}\nResistência: {resistencia}"

# ✅ Gerar informações de mercado
info = f"""
--- Gráfico Diário ---
{analisar(df_diario, 'Diário')}

--- Gráfico H1 --
{analisar(df_h1, 'H1')}

--- Gráfico M5 ---
{analisar(df_m5, 'M5')}
Número de correções: 3
Topo/Fundo Duplo: Não
Falha de Rompimento: Não
Última vela é Marubozu: Não
"""

# ✅ Formatar candles recentes
candles = "\n".join([
    f"Abertura: {row.open}, Fechamento: {row.close}, Máxima: {row.high}, Mínima: {row.low}"
    for _, row in df_m5.iterrows()
])

# ✅ Criar prompt para o modelo com chance em porcentagem
prompt = f"""
Sou um trader utilizando price action. Todas as decisões são feitas no gráfico M5.

{info}

Candles recentes:
{candles}

Com base nisso, responda em português e de forma organizada:
1. Devo comprar, vender ou aguardar?
2. Qual o melhor horário para entrar?
3. Onde devo posicionar o stop loss e o alvo da operação?
4. Qual a chance do trade dar certo? Responda em porcentagem (%).
5. Quanto tempo médio a operação deve demorar até atingir o alvo ou o stop?
"""

# ✅ Consultar modelo do Ollama
modelo = ChatOllama(model="llama3")
resposta = modelo.invoke(prompt)

# ✅ Exibir resposta
print("📈 Resposta da LLM:\n")
print(resposta.content)

📈 Resposta da LLM:

**Análise Price Action M5**

1. **Devo comprar, vender ou aguardar?**: Aguardar. O gráfico M5 está mostrando um canal estreito de baixa e não há uma oportunidade clara para compra ou venda.
2. **Qual o melhor horário para entrar?**: Nenhum é recomendado, pois a tendência atual é neutra e não há um momento específico que justifique uma operação.
3. **Onde devo posicionar o stop loss e o alvo da operação?**: Não há um ponto de entrada definido, portanto, não há como posicionar o stop loss e o alvo.
4. **Qual a chance do trade dar certo?**: Aproximadamente 50% (ou seja, indeterminado).
5. **Quanto tempo médio a operação deve demorar até atingir o alvo ou o stop?**: Não há um alvo definido e não há uma tendência clara para que possamos estimar um prazo.

Observações adicionais:

* O ciclo do gráfico M5 é um canal estreito de baixa, o que sugere uma tendência neutra.
* A Suporte e a Resistência estão próximas entre si, o que também indica uma falta de impulso para compra